In [1]:
future_num = 144 # 何足先を予測するか
feature_num = 5 # volume, open, high, low, closeの5項目
batch_size = 128
time_steps = 50 # lstmのtimesteps
moving_average_num = 500 # 移動平均を取るCandle数
n_epocs = 10 
# データをtrain, testに分割するIndex
val_idx_from = 80000
test_idx_from = 100000

lstm_hidden_dim = 16
target_dim = 1

In [2]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pickle
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)

In [3]:
# 1. CSVファイルの読み込み
df = pd.read_csv('../data/USD_JPY_201601-201908_M10.csv', index_col='Datetime')

# 2. 教師データの作成
future_price = df.iloc[future_num:]['Close'].values
curr_price = df.iloc[:-future_num]['Close'].values
y_data_tmp = future_price - curr_price
y_data = np.zeros_like(y_data_tmp)
y_data[y_data_tmp > 0] = 1
y_data = y_data[moving_average_num:]
# 3. 価格の正規化
cols = df.columns
for col in cols:
    df['Roll_' + col] = df[col].rolling(window=500, min_periods=500).mean()
    df[col] = df[col] / df['Roll_' + col] - 1

#最初の500足分は移動平均データがないため除く。後半の144足分は予測データがないため除く
X_data = df.iloc[moving_average_num:-future_num][cols].values

# 4. データの分割、TorchのTensorに変換
#学習用データ
X_train = torch.tensor(X_data[:val_idx_from], dtype=torch.float, device=device)
y_train = torch.tensor(y_data[:val_idx_from], dtype=torch.float, device=device)
#評価用データ
X_val   = torch.tensor(X_data[val_idx_from:test_idx_from], dtype=torch.float, device=device)
y_val   = y_data[val_idx_from:test_idx_from]
#テスト用データ
X_test  = torch.tensor(X_data[test_idx_from:], dtype=torch.float, device=device)
y_test  = y_data[test_idx_from:]

In [4]:
class LSTMClassifier(nn.Module):
    def __init__(self, lstm_input_dim, lstm_hidden_dim, target_dim):
        super(LSTMClassifier, self).__init__()
        self.input_dim = lstm_input_dim
        self.hidden_dim = lstm_hidden_dim
        self.lstm = nn.LSTM(input_size=lstm_input_dim, 
                            hidden_size=lstm_hidden_dim,
                            num_layers=1, #default
                            #dropout=0.2,
                            batch_first=True
                            )
        self.dense = nn.Linear(lstm_hidden_dim, target_dim)

    def forward(self, X_input):
        _, lstm_out = self.lstm(X_input)
        # LSTMの最終出力のみを利用する。
        linear_out = self.dense(lstm_out[0].view(X_input.size(0), -1))
        return torch.sigmoid(linear_out)

In [5]:
def prep_feature_data(batch_idx, time_steps, X_data, feature_num, device):
    feats = torch.zeros((len(batch_idx), time_steps, feature_num), dtype=torch.float, device=device)
    for b_i, b_idx in enumerate(batch_idx):
        # 過去のN足分をtime stepのデータとして格納する。
        b_slc = slice(b_idx + 1 - time_steps ,b_idx + 1)
        feats[b_i, :, :] = X_data[b_slc, :]

    return feats

In [6]:
# Prepare for training
model = LSTMClassifier(feature_num, lstm_hidden_dim, target_dim).to(device)
loss_function = nn.BCELoss()
optimizer= optim.Adam(model.parameters(), lr=1e-4)

In [7]:
train_size = X_train.size(0)
best_acc_score = 0
for epoch in range(n_epocs):
    # 1. まずはtrainデータのindexをランダムに入れ替える。最初のtime_steps分は使わない。
    perm_idx = np.random.permutation(np.arange(time_steps, train_size))
    # 2. batch size毎にperm_idxの対象のindexを取得
    for t_i in range(0, len(perm_idx), batch_size):
        batch_idx = perm_idx[t_i:(t_i + batch_size)]
        # 3. LSTM入力用の時系列データの準備
        feats = prep_feature_data(batch_idx, time_steps, X_train, feature_num, device)
        y_target = y_train[batch_idx]
        # 4. pytorch LSTMの学習実施
        model.zero_grad()
        train_scores = model(feats) # batch size x time steps x feature_num
        loss = loss_function(train_scores, y_target.view(-1, 1))
        loss.backward()
        optimizer.step()

    # 5. validationデータの評価
    print('EPOCH: ', str(epoch), ' loss :', loss.item())
    with torch.no_grad():
        feats_val = prep_feature_data(np.arange(time_steps, X_val.size(0)), time_steps, X_val, feature_num, device)
        val_scores = model(feats_val)
        tmp_scores = val_scores.view(-1).to('cpu').numpy()
        bi_scores = np.round(tmp_scores)
        acc_score = accuracy_score(y_val[time_steps:], bi_scores)
        roc_score = roc_auc_score(y_val[time_steps:], tmp_scores)
        print('Val ACC Score :', acc_score, ' ROC AUC Score :', roc_score)

    # 6. validationの評価が良ければモデルを保存
    if acc_score > best_acc_score:
        best_acc_score = acc_score
        torch.save(model.state_dict(),'./pytorch_v1.mdl')
        print('best score updated, Pytorch model was saved!!', )

# 7. bestモデルで予測する。
model.load_state_dict(torch.load('./pytorch_v1.mdl'))
with torch.no_grad():
    feats_test = prep_feature_data(np.arange(time_steps, X_test.size(0)), time_steps, X_test, feature_num, device)
    val_scores = model(feats_test)
    tmp_scores = val_scores.view(-1).to('cpu').numpy()
    bi_scores = np.round(tmp_scores)
    acc_score = accuracy_score(y_test[time_steps:], bi_scores)
    roc_score = roc_auc_score(y_test[time_steps:], tmp_scores)
    print('Test ACC Score :', acc_score, ' ROC AUC Score :', roc_score)

EPOCH:  0  loss : 0.692438006401062
Val ACC Score : 0.45729323308270675  ROC AUC Score : 0.4653708720407381
best score updated, Pytorch model was saved!!
EPOCH:  1  loss : 0.6937869787216187
Val ACC Score : 0.45669172932330826  ROC AUC Score : 0.46604893518135393
EPOCH:  2  loss : 0.6911344528198242
Val ACC Score : 0.4641102756892231  ROC AUC Score : 0.4690256528753118
best score updated, Pytorch model was saved!!
EPOCH:  3  loss : 0.6901702880859375
Val ACC Score : 0.46461152882205514  ROC AUC Score : 0.494518581332261
best score updated, Pytorch model was saved!!
EPOCH:  4  loss : 0.6916045546531677
Val ACC Score : 0.47218045112781953  ROC AUC Score : 0.5252373680624068
best score updated, Pytorch model was saved!!
EPOCH:  5  loss : 0.6945702433586121
Val ACC Score : 0.4651127819548872  ROC AUC Score : 0.5352784009690617
EPOCH:  6  loss : 0.6921018958091736
Val ACC Score : 0.4780952380952381  ROC AUC Score : 0.5377794198366386
best score updated, Pytorch model was saved!!
EPOCH:  7  

In [8]:
from backtesting import Strategy
from backtesting import Backtest
df = pd.read_csv('../data/USD_JPY_201601-201908_M10.csv', index_col='Datetime')
df.index = pd.to_datetime(df.index)

C:\ProgramData\Anaconda3\envs\AI-GPU\lib\site-packages\backtesting\_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [13]:
class myLSTMStrategy(Strategy):
    def init(self):
        # 1. LSTMの学習済みモデルの読み込み
        self.model = LSTMClassifier(feature_num, lstm_hidden_dim, target_dim).to(device)
        # load model
        self.model.load_state_dict(torch.load('./pytorch_v1.mdl'))

    def next(self): 
        # 過去500ステップ分のデータが貯まるまではスキップ
        # 1日に1回のみ取引するため、hour & minuteが0の時のみ処理するようにする。
        if len(self.data) >= moving_average_num + time_steps and len(self.data) % future_num == 0:
            # 2. 推測用データの用意
            x_array = self.prepare_data()
            x_tensor = torch.tensor(x_array, dtype=torch.float, device=device)
            # 3. 予測の実行
            with torch.no_grad():
                y_pred = self.predict(x_tensor.view(1, time_steps, feature_num))

            # 4. 予測が買い(1)であればbuy()、それ以外はsell()
            if y_pred == 1:
                self.buy(sl=self.data.Close[-1]*0.99, 
                         tp=self.data.Close[-1]*1.01)
            else:
                self.sell(sl=self.data.Close[-1]*1.01, 
                         tp=self.data.Close[-1]*0.99)

    def prepare_data(self):
        # いったんPandasのデータフレームに変換
        tmp_df = pd.concat([
                    self.data.Volume.to_series(), 
                    self.data.Open.to_series(), 
                    self.data.High.to_series(), 
                    self.data.Low.to_series(), 
                    self.data.Close.to_series(), 
                    ], axis=1)

        # 500足の移動平均に対する割合とする。
        cols = tmp_df.columns
        for col in cols:
            tmp_df['Roll_' + col] = tmp_df[col].rolling(window=moving_average_num, min_periods=moving_average_num).mean()
            tmp_df[col] = tmp_df[col] / tmp_df['Roll_' + col] - 1

        #最後のtime_steps分のみの値を返す
        return tmp_df.tail(time_steps)[cols].values

    def predict(self, x_array):
        y_score = self.model(x_array) 
        return np.round(y_score.view(-1).to('cpu').numpy())[0]

In [14]:
bt = Backtest(df[100000:], myLSTMStrategy, cash=100000, commission=.00004)
bt.run()

Start                     2018-09-06 13:10:00
End                       2019-08-01 03:50:00
Duration                    328 days 14:40:00
Exposure [%]                          97.6289
Equity Final [$]                       100968
Equity Peak [$]                        103115
Return [%]                           0.967775
Buy & Hold Return [%]                 1.76702
Max. Drawdown [%]                     -4.0093
Avg. Drawdown [%]                   -0.247101
Max. Drawdown Duration      239 days 14:20:00
Avg. Drawdown Duration        5 days 13:21:00
# Trades                                  228
Win Rate [%]                          50.4386
Best Trade [%]                       0.998397
Worst Trade [%]                      -1.00666
Avg. Trade [%]                     0.00745971
Max. Trade Duration           3 days 01:00:00
Avg. Trade Duration           1 days 09:47:00
Expectancy [%]                       0.252935
SQN                                  0.322832
Sharpe Ratio                      